In [61]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [62]:
df=pd.read_csv('data.csv',header=0,usecols=[0])
df

,国名
0,アメリカ
1,中国
2,イギリス
3,ロシア
4,ドイツ
5,カナダ


In [63]:
DMUs=df.values
print(DMUs)

[['アメリカ']
 ['中国']
 ['イギリス']
 ['ロシア']
 ['ドイツ']
 ['カナダ']]


In [64]:
dfx=pd.read_csv('data.csv',header=0,index_col=0,usecols=[0,1,2])
dfx

,人口(百万人),GDP
国名,,
アメリカ,321.774,14682.739
中国,1376.049,5320.232
イギリス,64.716,2676.520
ロシア,143.457,999.832
ドイツ,80.689,3226.726
カナダ,35.940,1359.773


In [65]:
x=dfx

In [66]:
print(dfx)

       人口(百万人)        GDP
国名                       
アメリカ   321.774  14682.739
中国    1376.049   5320.232
イギリス    64.716   2676.520
ロシア    143.457    999.832
ドイツ     80.689   3226.726
カナダ     35.940   1359.773


In [67]:
x.shape

(6, 2)

In [68]:
dfy=pd.read_csv('data.csv',header=0,usecols=[3,4,5])
dfy

,メダル獲得数 金,銀,銅
0,46,37,38
1,26,18,26
2,27,23,17
3,19,18,19
4,17,10,15
5,4,3,15


In [69]:
y=dfy

In [70]:
y.shape

(6, 3)

In [72]:
n,m=x.shape
n,s=y.shape

In [73]:
res=[]

In [74]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    us=np.array([u[i].varValue for i in range(s)]) #u*
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs: ',vs)
    print('us: ',us)
    print(' ')
#     # 参照集合作成
#     (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)

#     res.append([DMUs[o],value(prob.objective),
#                 set(eo),tuple(vs),tuple(us)])

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['アメリカ']
vs:  [0.00310777 0.        ]
us:  [0.00331766 0.         0.00656151]
 
DMU_1 :  ['中国']
vs:  [0.         0.00018796]
us:  [0.         0.         0.00989106]
 
DMU_2 :  ['イギリス']
vs:  [0.01545213 0.        ]
us:  [0.01649572 0.         0.03262444]
 
DMU_3 :  ['ロシア']
vs:  [0.         0.00100017]
us:  [0.         0.         0.05263158]
 
DMU_4 :  ['ドイツ']
vs:  [0.00372388 0.00021679]
us:  [0.01493177 0.         0.02459301]
 
DMU_5 :  ['カナダ']
vs:  [0.00936054 0.00048801]
us:  [0.         0.03972503 0.05872166]
 


In [75]:
print(res)

[[array(['アメリカ'], dtype=object), 0.40194980579999995], [array(['中国'], dtype=object), 0.257167573], [array(['イギリス'], dtype=object), 0.9999999980000001], [array(['ロシア'], dtype=object), 1.0000000009999999], [array(['ドイツ'], dtype=object), 0.622735255], [array(['カナダ'], dtype=object), 1.000000005]]


In [76]:
df_f=pd.DataFrame(res,columns=['国名','効率値'])
df_s=df_f.sort_values('効率値',ascending=False)
df_s

,国名,効率値
5,[カナダ],1.000000
3,[ロシア],1.000000
2,[イギリス],1.000000
4,[ドイツ],0.622735
0,[アメリカ],0.401950
1,[中国],0.257168


In [71]:
# def DEA_CCR(x,y,DMUs):
#     n,m=x.shape
#     n,s=y.shape
    
#     res=[]
    
#     for o in range(n):
#         prob=LpProblem('DMU_'+str(o),LpMaximize)
#         #　重み
#         v=[LpVariable('v'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(m)]
#         u=[LpVariable('u'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(s)]
#         #　目的関数
#         prob += lpDot(u,y.iloc[:,o])

#         # 制約条件
#         prob += lpDot(v,x.iloc[:,o])==1,'Normalize'+str(o)

#         for j in range(n):
#             prob += lpDot(u,y.iloc[:,j]) <= lpDot(v,x.iloc[:,j])

#         prob.solve()
#         vs=np.array([v[i].varValue for i in range(m)]) #v*
#         us=np.array([v[i].varValue for i in range(m)]) #u*
#         # 参照集合作成
#         (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)
#         res.append([DMUs[o],value(prob.objective),
#                     set(eo),tuple(vs),tuple(us)])
#     return res